In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = '/content/drive/MyDrive/Dataset/'

In [1]:
import pandas as pd

In [2]:
positive_df = pd.read_csv('positive_sample_7500_1.csv', header=None)
negative_df = pd.read_csv('negative_sample_7500_1.csv', header=None)

In [3]:
def load_protein_sequences(file_path):
    protein_dict = {}
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                protein_dict[parts[0]] = parts[1]
    return protein_dict
protein_sequences = load_protein_sequences('protein_sequences.txt')

In [4]:
positive_df_2 = pd.DataFrame([positive_df[1],positive_df[0]]).transpose()
negative_df_2 = pd.DataFrame([negative_df[1],negative_df[0]]).transpose()

In [5]:
positive_df = pd.concat([positive_df,positive_df_2])
negative_df = pd.concat([negative_df,negative_df_2])

In [6]:
positive_df[2] = 1
negative_df[2] = 0
data = pd.concat([positive_df, negative_df],ignore_index=True)

In [7]:
data = data.sample(frac=1,random_state=42).reset_index(drop=True)

In [8]:
data[0] = [protein_sequences[x] for x in data[0]]
data[1] = [protein_sequences[x] for x in data[1]]

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
sequences1 = data[0].values  # Amino acid sequences of protein 1
sequences2 = data[1].values  # Amino acid sequences of protein 2
labels = data[2].values     # Interaction labels (0 or 1)

In [11]:
tokenizer = Tokenizer(char_level=True)  # Tokenize at character level
tokenizer.fit_on_texts(sequences1 + sequences2)

# Convert sequences to numerical tokens
encoded_sequences1 = tokenizer.texts_to_sequences(sequences1)
encoded_sequences2 = tokenizer.texts_to_sequences(sequences2)

# Pad sequences to a fixed length (choose an appropriate maxlen)
maxlen = 100  # Example max length, adjust as needed
padded_sequences1 = pad_sequences(encoded_sequences1, maxlen=maxlen, padding='post')
padded_sequences2 = pad_sequences(encoded_sequences2, maxlen=maxlen, padding='post')

In [13]:
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define the number of folds
k = 5  # You can adjust this value

# Create KFold object
kf = KFold(n_splits=k, shuffle=True, random_state=42)  # Shuffle and set random_state for reproducibility

# Lists to store evaluation results
accuracies = []
precisions = []
recalls = []
f1_scores = []

# Iterate through folds
for fold, (train_index, test_index) in enumerate(kf.split(padded_sequences1)):  # Assuming padded_sequences1 and padded_sequences2 have the same length
    print(f"Fold {fold + 1}")

    # Split data into training and testing sets for this fold
    train_sequences1_fold, test_sequences1_fold = padded_sequences1[train_index], padded_sequences1[test_index]
    train_sequences2_fold, test_sequences2_fold = padded_sequences2[train_index], padded_sequences2[test_index]
    train_labels_fold, test_labels_fold = labels[train_index], labels[test_index]

    # Create and compile the model (same as before)
    vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size for embedding layer

    # Input layers for protein sequences
    input1 = Input(shape=(maxlen,))
    input2 = Input(shape=(maxlen,))

    # Embedding layers to represent amino acids as vectors
    embedding_layer = Embedding(vocab_size, 128)  # 128-dimensional embeddings
    embedded_sequences1 = embedding_layer(input1)
    embedded_sequences2 = embedding_layer(input2)

    # LSTM layers to process the sequences
    lstm_layer = LSTM(64)  # 64 LSTM units
    lstm_output1 = lstm_layer(embedded_sequences1)
    lstm_output2 = lstm_layer(embedded_sequences2)

    # Concatenate LSTM outputs
    merged = concatenate([lstm_output1, lstm_output2])

    # Dense layers for classification
    output = Dense(1, activation='sigmoid')(merged)

    # Create the model
    model = Model(inputs=[input1, input2], outputs=output)

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model on this fold's training data
    model.fit([train_sequences1_fold, train_sequences2_fold], train_labels_fold, epochs=100, batch_size=2500)

    # Evaluate the model on this fold's testing data
    predictions = model.predict([test_sequences1_fold, test_sequences2_fold])
    predicted_labels_fold = (predictions > 0.5).astype(int)

    accuracy = accuracy_score(test_labels_fold, predicted_labels_fold)
    precision = precision_score(test_labels_fold, predicted_labels_fold)
    recall = recall_score(test_labels_fold, predicted_labels_fold)
    f1 = f1_score(test_labels_fold, predicted_labels_fold)

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}")

# Calculate average accuracy and loss across all folds
average_accuracy = np.mean(accuracies)
average_precision = np.mean(precisions)
average_recalls = np.mean(recalls)
average_f1_scores = np.mean(f1_scores)

print(f"Average Accuracy: {average_accuracy:.4f}")
print(f"Average Precision: {average_precision:.4f}")
print(f"Average Recalls: {average_recalls:.4f}")
print(f"Average F1 Scores: {average_f1_scores:.4f}")


Fold 1
Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5193 - loss: 0.6923
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5505 - loss: 0.6859
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5694 - loss: 0.6865
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5660 - loss: 0.6814
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5673 - loss: 0.6799
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5737 - loss: 0.6749
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5935 - loss: 0.6680
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6039 - loss: 0.6656
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.6039 - loss: 0.6628
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5996 - loss: 0.6660
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5997 - loss: 0.6622
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - ac